In [3]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import EAtools as ea
import pandas as pd
import os   
import sys 
import numpy
import fileinput
import matplotlib.pyplot as plt 
import matplotlib.ticker  
from matplotlib.ticker import FormatStrFormatter 
import pandas.io.sql
import pyodbc 
from matplotlib.pyplot import figure, show
from matplotlib.patches import Ellipse
import numpy as np
from pandas import *
import EAtools as ea
ea.set_options()
ea.ea_report_style()
from pylab import *
import matplotlib.ticker as tkr
from datetime import datetime, date, time, timedelta
ea_p=ea.ea_p
ea_s=ea.ea_s
def cm2inch(value):
    return value/2.54
import pyarrow as pa
import pyarrow.parquet as pq

In [4]:
from db import DB
import os
ea.set_options()
ea.ea_report_style()
%pylab inline
%matplotlib inline
path='/media/usb/notebooks/Julia/20192020_review/'
db = DB(profile="hallj") 
#db = DB(profile="millerr") 

Populating the interactive namespace from numpy and matplotlib


/home/julia/.local/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime', 'fft', 'linalg', 'random', 'info', 'power']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# create parquet file with offered reserves

In [3]:
#do just for Mercury
#RTM best not to group. So bring out ProductSubClass and ProductCategory. 
#This ensures you get qunatoty weighted over mean over FIR and SIR, PLS and TWD 
Q=r"""
    SELECT TradingDate
          ,TradingPeriod
          ,PointOfConnection
          ,Tranche
          ,ProductType
          ,ProductSubClass
          ,ProductCategory
          --,sum(Megawatt) as Megawatt
          --,avg(DollarsPerMegawattHour) as DollarsPerMegawattHour
          ,Megawatt -- RTM
          ,DollarsPerMegawattHour -- RTM
      FROM [ElectricityAuthorityData].[wholesale].[Offers]
      where TradingDate>='2019-01-01' and PointOfConnection in ('ARI1101','ARI1102','ARA2201','ATI2201','KPO1101','MTI2201','OHK2201','WPA2201','WKM2201') and Unit in ('ARI0','ARA0','ATI0','KPO0','MTI0','OHK0','WPA0','WKM0')  
      and ProductClass='Injection' and ProductType in ('Reserve','Energy') and IsLatest='Y'
      --group by TradingDate, TradingPeriod, PointOfConnection, Tranche, ProductType -- RTM
      order by TradingDate, TradingPeriod, PointOfConnection, Tranche, ProductType, ProductSubClass, ProductCategory -- RTM added ProductSubClass and ProductCategory
"""

ReserveOffers= db.query(Q)
ReserveOffers.tail()

,TradingDate,TradingPeriod,PointOfConnection,Tranche,ProductType,ProductSubClass,ProductCategory,Megawatt,DollarsPerMegawattHour
7263517,2021-09-15,48,WPA2201,3,Reserve,FIR,TWD,0.0,0.03
7263518,2021-09-15,48,WPA2201,3,Reserve,SIR,PLS,0.0,545.00
7263519,2021-09-15,48,WPA2201,3,Reserve,SIR,TWD,0.0,0.03
7263520,2021-09-15,48,WPA2201,4,Energy,N/A,N/A,0.0,200.00
7263521,2021-09-15,48,WPA2201,5,Energy,N/A,N/A,36.0,555.00


In [4]:
ReserveOffers['TradingDate'] = pd.to_datetime(ReserveOffers.TradingDate)
ReserveOffers.head()

,TradingDate,TradingPeriod,PointOfConnection,Tranche,ProductType,ProductSubClass,ProductCategory,Megawatt,DollarsPerMegawattHour
0,2019-01-01,1,ARA2201,1,Energy,N/A,N/A,22.0,0.00
1,2019-01-01,1,ARA2201,1,Reserve,FIR,PLS,0.0,0.01
2,2019-01-01,1,ARA2201,1,Reserve,FIR,TWD,0.0,0.00
3,2019-01-01,1,ARA2201,1,Reserve,SIR,PLS,0.0,0.01
4,2019-01-01,1,ARA2201,1,Reserve,SIR,TWD,0.0,0.00


In [5]:
#decided to leave the percent as the total of FIR and SIR (after discussion with Roger)
Offerspercent = ReserveOffers.copy()
Offerspercent = pd.DataFrame(Offerspercent.groupby(['TradingDate','TradingPeriod','ProductType'])['Megawatt'].sum())
Offerspercent.head()

Megawatt
TradingDate TradingPeriod ProductType          
2019-01-01  1             Energy          940.0
                          Reserve         182.0
            2             Energy          940.0
                          Reserve         203.0
            3             Energy          940.0

In [6]:
OffersPercent_v2 = Offerspercent.unstack(level=2)
OffersPercent_v2.head()

Megawatt        
ProductType                 Energy Reserve
TradingDate TradingPeriod                 
2019-01-01  1                940.0   182.0
            2                940.0   203.0
            3                940.0   210.0
            4                940.0   203.0
            5                940.0   190.0

In [7]:
OffersPercent_v2['percent'] = (OffersPercent_v2[('Megawatt','Reserve')]/(OffersPercent_v2[('Megawatt','Reserve')]+OffersPercent_v2[('Megawatt','Energy')]))*100
OffersPercent_v2.head()

Megawatt            percent
ProductType                 Energy Reserve           
TradingDate TradingPeriod                            
2019-01-01  1                940.0   182.0  16.221034
            2                940.0   203.0  17.760280
            3                940.0   210.0  18.260870
            4                940.0   203.0  17.760280
            5                940.0   190.0  16.814159

In [8]:
waikato_table = pa.Table.from_pandas(OffersPercent_v2)
pq.write_table(waikato_table, path + "waikato_reserves_percent.parquet")

In [9]:
ReserveOffers = ReserveOffers[ReserveOffers.ProductType=='Reserve']
ReserveOffers['MWtimesdollars'] = ReserveOffers['Megawatt']*ReserveOffers['DollarsPerMegawattHour']
ReserveOffers.head()

,TradingDate,TradingPeriod,PointOfConnection,Tranche,ProductType,ProductSubClass,ProductCategory,Megawatt,DollarsPerMegawattHour,MWtimesdollars
1,2019-01-01,1,ARA2201,1,Reserve,FIR,PLS,0.0,0.01,0.0
2,2019-01-01,1,ARA2201,1,Reserve,FIR,TWD,0.0,0.00,0.0
3,2019-01-01,1,ARA2201,1,Reserve,SIR,PLS,0.0,0.01,0.0
4,2019-01-01,1,ARA2201,1,Reserve,SIR,TWD,0.0,0.00,0.0
6,2019-01-01,1,ARA2201,2,Reserve,FIR,PLS,0.0,5.00,0.0


In [10]:
ReserveOffers.set_index(['PointOfConnection','TradingDate','TradingPeriod','Tranche','ProductSubClass','ProductCategory'],inplace=True)
ReserveOffers.head()

ProductType  Megawatt  DollarsPerMegawattHour  MWtimesdollars
PointOfConnection TradingDate TradingPeriod Tranche ProductSubClass ProductCategory                                                              
ARA2201           2019-01-01  1             1       FIR             PLS                 Reserve       0.0                    0.01             0.0
                                                                    TWD                 Reserve       0.0                    0.00             0.0
                                                    SIR             PLS                 Reserve       0.0                    0.01             0.0
                                                                    TWD                 Reserve       0.0                    0.00             0.0
                                            2       FIR             PLS                 Reserve       0.0                    5.00             0.0

In [11]:
#temp1 = pd.DataFrame(ReserveOffers.groupby(level=[0,1,2])['MWtimesdollars'].sum())
#temp2 = pd.DataFrame(ReserveOffers.groupby(level=[0,1,2])['Megawatt'].sum())
#temp3 = temp1.join(temp2)
temp3 = ReserveOffers.groupby(level=['TradingDate', 'TradingPeriod']).agg({'MWtimesdollars': sum, 'Megawatt': sum}) #RTM
temp3.head()

Megawatt  MWtimesdollars
TradingDate TradingPeriod                          
2019-01-01  1                 182.0       274656.94
            2                 203.0       382456.93
            3                 210.0       367845.35
            4                 203.0       367785.72
            5                 190.0       328550.87

In [12]:
temp3['WeightstarPrice'] = temp3['MWtimesdollars']/temp3['Megawatt']

In [13]:
temp3 = temp3[['WeightstarPrice']]
temp3.head()

WeightstarPrice
TradingDate TradingPeriod                 
2019-01-01  1                  1509.104066
            2                  1884.024286
            3                  1751.644524
            4                  1811.752315
            5                  1729.215105

In [14]:
#waikato = pd.DataFrame(temp3.groupby(level=[1,2])['WeightstarPrice'].mean())
waikato = temp3.copy()

In [15]:
waikato = waikato.fillna(0)

In [16]:
waikato_table = pa.Table.from_pandas(waikato)

In [17]:
pq.write_table(waikato_table, path + "waikato_reserves_QWOP.parquet")

# create parquet file with QWOPs

In [3]:
offers = pd.read_parquet(path+'/rogersdata.parquet')
offers.head()

,PointOfConnection,Unit,TradingDate,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2
0,ARA2201,ARA0,2014-01-01,1,1,22.0,0.0,TAUPO,TAUPO
1,ARA2201,ARA0,2014-01-01,1,4,4.0,94.0,TAUPO,TAUPO
2,ARA2201,ARA0,2014-01-01,1,5,52.0,165.0,TAUPO,TAUPO
3,ARA2201,ARA0,2014-01-01,2,1,22.0,0.0,TAUPO,TAUPO
4,ARA2201,ARA0,2014-01-01,2,4,4.0,94.0,TAUPO,TAUPO


In [4]:
offers['MWtimesdollars'] = offers['Megawatt']*offers['DollarsPerMegawattHour']
offers.head()

,PointOfConnection,Unit,TradingDate,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2,MWtimesdollars
0,ARA2201,ARA0,2014-01-01,1,1,22.0,0.0,TAUPO,TAUPO,0.0
1,ARA2201,ARA0,2014-01-01,1,4,4.0,94.0,TAUPO,TAUPO,376.0
2,ARA2201,ARA0,2014-01-01,1,5,52.0,165.0,TAUPO,TAUPO,8580.0
3,ARA2201,ARA0,2014-01-01,2,1,22.0,0.0,TAUPO,TAUPO,0.0
4,ARA2201,ARA0,2014-01-01,2,4,4.0,94.0,TAUPO,TAUPO,376.0


In [5]:
offers.set_index(['NewPointOfConnection','Unit','TradingDate','TradingPeriod','Tranche'],inplace=True)
offers.head()

PointOfConnection  Megawatt  DollarsPerMegawattHour NewPointOfConnection2  MWtimesdollars
NewPointOfConnection Unit TradingDate TradingPeriod Tranche                                                                                          
TAUPO                ARA0 2014-01-01  1             1                 ARA2201      22.0                     0.0                 TAUPO             0.0
                                                    4                 ARA2201       4.0                    94.0                 TAUPO           376.0
                                                    5                 ARA2201      52.0                   165.0                 TAUPO          8580.0
                                      2             1                 ARA2201      22.0                     0.0                 TAUPO             0.0
                                                    4                 ARA2201       4.0                    94.0                 TAUPO           376.0

In [6]:
temp1 = pd.DataFrame(offers.groupby(level=[0,2,3])['MWtimesdollars'].sum())
temp2 = pd.DataFrame(offers.groupby(level=[0,2,3])['Megawatt'].sum())
temp3 = temp1.join(temp2)
temp3.head()

MWtimesdollars  Megawatt
NewPointOfConnection TradingDate TradingPeriod                          
CYD                  2014-01-01  1                    12286.81     395.0
                                 2                    12286.81     395.0
                                 3                    12286.81     395.0
                                 4                    12286.81     395.0
                                 5                    12286.81     395.0

In [7]:
offers = offers.reset_index().set_index(['NewPointOfConnection2','Unit','TradingDate','TradingPeriod','Tranche'])
offers.head()

NewPointOfConnection PointOfConnection  Megawatt  DollarsPerMegawattHour  MWtimesdollars
NewPointOfConnection2 Unit TradingDate TradingPeriod Tranche                                                                                         
TAUPO                 ARA0 2014-01-01  1             1                      TAUPO           ARA2201      22.0                     0.0             0.0
                                                     4                      TAUPO           ARA2201       4.0                    94.0           376.0
                                                     5                      TAUPO           ARA2201      52.0                   165.0          8580.0
                                       2             1                      TAUPO           ARA2201      22.0                     0.0             0.0
                                                     4                      TAUPO           ARA2201       4.0                    94.0           376.0

In [8]:
temp1b = pd.DataFrame(offers.groupby(level=[0,2,3])['MWtimesdollars'].sum())
temp2b = pd.DataFrame(offers.groupby(level=[0,2,3])['Megawatt'].sum())
temp3b = temp1b.join(temp2b)
temp3b.head()

MWtimesdollars  Megawatt
NewPointOfConnection2 TradingDate TradingPeriod                          
CLUTHA                2014-01-01  1                    18371.55    670.01
                                  2                    18371.55    670.01
                                  3                    18371.55    670.01
                                  4                    15971.55    645.01
                                  5                    15971.55    645.01

In [9]:
temp3['WeightstarPrice'] = temp3['MWtimesdollars']/temp3['Megawatt']
temp3b['WeightstarPrice'] = temp3b['MWtimesdollars']/temp3b['Megawatt']
temp3.head()

MWtimesdollars  Megawatt  WeightstarPrice
NewPointOfConnection TradingDate TradingPeriod                                           
CYD                  2014-01-01  1                    12286.81     395.0        31.105848
                                 2                    12286.81     395.0        31.105848
                                 3                    12286.81     395.0        31.105848
                                 4                    12286.81     395.0        31.105848
                                 5                    12286.81     395.0        31.105848

In [10]:
temp3 = temp3.reset_index().set_index(['TradingDate','TradingPeriod'])
temp3 = temp3[['NewPointOfConnection','WeightstarPrice']]
temp3.head()

NewPointOfConnection  WeightstarPrice
TradingDate TradingPeriod                                      
2014-01-01  1                              CYD        31.105848
            2                              CYD        31.105848
            3                              CYD        31.105848
            4                              CYD        31.105848
            5                              CYD        31.105848

In [11]:
temp3b = temp3b.reset_index().set_index(['TradingDate','TradingPeriod'])
temp3b = temp3b[['NewPointOfConnection2','WeightstarPrice']]
temp3b.head()

NewPointOfConnection2  WeightstarPrice
TradingDate TradingPeriod                                       
2014-01-01  1                            CLUTHA        27.419815
            2                            CLUTHA        27.419815
            3                            CLUTHA        27.419815
            4                            CLUTHA        24.761709
            5                            CLUTHA        24.761709

In [12]:
allthermal = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['HUNTLY','STRATFORD','MKE','WHI','JRD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
#allgasthermal = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['HUNTLY','MKE','STRATFORD','JRD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
#do gas excluding huntly
allgasthermal = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['MKE','STRATFORD','JRD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
#do gas excluding just rankines at huntly
allgasthermal_v2 = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['E3P','HLY6','TCC','PEAKERS','MKE','JRD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
huntly = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['HUNTLY'])].groupby(level=[0,1])['WeightstarPrice'].mean())
stratford = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['STRATFORD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
mckee = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['MKE'])].groupby(level=[0,1])['WeightstarPrice'].mean())
whirinaki = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['WHI'])].groupby(level=[0,1])['WeightstarPrice'].mean())
junctionrd = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['JRD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
rankines = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['RANKINES'])].groupby(level=[0,1])['WeightstarPrice'].mean())
e3p = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['E3P'])].groupby(level=[0,1])['WeightstarPrice'].mean())
huntly_unit6 = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['HLY6'])].groupby(level=[0,1])['WeightstarPrice'].mean())
stratford_peakers = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['PEAKERS'])].groupby(level=[0,1])['WeightstarPrice'].mean())
TCC = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['TCC'])].groupby(level=[0,1])['WeightstarPrice'].mean())
allhydro = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['MAN','WAITAKI','CLUTHA','TKA','TAUPO','WAIKAREMOANA'])].groupby(level=[0,1])['WeightstarPrice'].mean())
allSIhydro = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['MAN','WAITAKI','CLUTHA','TKA'])].groupby(level=[0,1])['WeightstarPrice'].mean())
allNIhydro = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['TAUPO','WAIKAREMOANA','TONGARIRO'])].groupby(level=[0,1])['WeightstarPrice'].mean())
waitaki = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['WAITAKI'])].groupby(level=[0,1])['WeightstarPrice'].mean())
waikato = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['TAUPO'])].groupby(level=[0,1])['WeightstarPrice'].mean())
clutha = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['CLUTHA'])].groupby(level=[0,1])['WeightstarPrice'].mean())
tekapo = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['TKA'])].groupby(level=[0,1])['WeightstarPrice'].mean())
manapouri = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['MAN'])].groupby(level=[0,1])['WeightstarPrice'].mean())
waikaremoana = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['WAIKAREMOANA'])].groupby(level=[0,1])['WeightstarPrice'].mean())
tongariro = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['TONGARIRO'])].groupby(level=[0,1])['WeightstarPrice'].mean())

In [ ]:
waitaki.head()

In [13]:
#sometimes there are zero megawatts for any tranche.
allthermal = allthermal.fillna(0)
allgasthermal = allgasthermal.fillna(0)
allgasthermal_v2 = allgasthermal_v2.fillna(0)
huntly = huntly.fillna(0)
stratford = stratford.fillna(0)
mckee = mckee.fillna(0)
whirinaki = whirinaki.fillna(0)
junctionrd = junctionrd.fillna(0)
rankines = rankines.fillna(0)
e3p = e3p.fillna(0)
huntly_unit6 = huntly_unit6.fillna(0)
stratford_peakers = stratford_peakers.fillna(0)
TCC = TCC.fillna(0)
allhydro = allhydro.fillna(0)
allSIhydro = allSIhydro.fillna(0)
allNIhydro = allNIhydro.fillna(0)
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
clutha = clutha.fillna(0)
tekapo = tekapo.fillna(0)
manapouri = manapouri.fillna(0)
waikaremoana = waikaremoana.fillna(0)
tongariro = tongariro.fillna(0)

In [14]:
#save parquet files
allthermal_table = pa.Table.from_pandas(allthermal)
allgasthermal_table = pa.Table.from_pandas(allgasthermal)
allgasthermal_v2_table = pa.Table.from_pandas(allgasthermal_v2)
huntly_table = pa.Table.from_pandas(huntly)
stratford_table = pa.Table.from_pandas(stratford)
mckee_table = pa.Table.from_pandas(mckee)
whirinaki_table = pa.Table.from_pandas(whirinaki)
junctionrd_table = pa.Table.from_pandas(junctionrd)
rankines_table = pa.Table.from_pandas(rankines)
e3p_table = pa.Table.from_pandas(e3p)
huntly_unit6_table = pa.Table.from_pandas(huntly_unit6)
stratford_peakers_table = pa.Table.from_pandas(stratford_peakers)
TCC_table = pa.Table.from_pandas(TCC)
allhydro_table = pa.Table.from_pandas(allhydro)
allSIhydro_table = pa.Table.from_pandas(allSIhydro)
allNIhydro_table = pa.Table.from_pandas(allNIhydro)
waitaki_table = pa.Table.from_pandas(waitaki)
waikato_table = pa.Table.from_pandas(waikato)
clutha_table = pa.Table.from_pandas(clutha)
tekapo_table = pa.Table.from_pandas(tekapo)
manapouri_table = pa.Table.from_pandas(manapouri)
waikaremoana_table = pa.Table.from_pandas(waikaremoana)
tongariro_table = pa.Table.from_pandas(tongariro)

In [15]:
pq.write_table(allthermal_table, path + "allthermal_v2.parquet")
pq.write_table(allgasthermal_table, path + "allgasthermal_v2.parquet")
pq.write_table(allgasthermal_v2_table, path + "allgasthermal_v2_v2.parquet")
pq.write_table(huntly_table, path + "huntly_v2.parquet")
pq.write_table(stratford_table, path + "stratford_v2.parquet")
pq.write_table(mckee_table, path + "mckee_v2.parquet")
pq.write_table(whirinaki_table, path + "whirinaki_v2.parquet")
pq.write_table(junctionrd_table, path + "junctionrd_v2.parquet")
pq.write_table(rankines_table, path + "rankines_v2.parquet")
pq.write_table(e3p_table, path + "e3p_v2.parquet")
pq.write_table(huntly_unit6_table, path + "huntly_unit6_v2.parquet")
pq.write_table(stratford_peakers_table, path + "stratford_peakers_v2.parquet")
pq.write_table(TCC_table, path + "TCC_v2.parquet")
pq.write_table(allhydro_table, path + "allhydro_v2.parquet")
pq.write_table(allSIhydro_table, path + "allSIhydro_v2.parquet")
pq.write_table(allNIhydro_table, path + "allNIhydro_v2.parquet")
pq.write_table(waitaki_table, path + "waitaki_v2.parquet")
pq.write_table(waikato_table, path + "waikato_v2.parquet")
pq.write_table(clutha_table, path + "clutha_v2.parquet")
pq.write_table(tekapo_table, path + "tekapo_v2.parquet")
pq.write_table(manapouri_table, path + "manapouri_v2.parquet")
pq.write_table(waikaremoana_table, path + "waikaremoana_v2.parquet")
pq.write_table(tongariro_table, path + "tongariro_v2.parquet")

# create parquet file with QWOPs with offers over $300 removed

In [5]:
offers = pd.read_parquet(path+'/rogersdata.parquet')
offers.head()

,PointOfConnection,Unit,TradingDate,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2
0,ARA2201,ARA0,2014-01-01,1,1,22.0,0.0,TAUPO,TAUPO
1,ARA2201,ARA0,2014-01-01,1,4,4.0,94.0,TAUPO,TAUPO
2,ARA2201,ARA0,2014-01-01,1,5,52.0,165.0,TAUPO,TAUPO
3,ARA2201,ARA0,2014-01-01,2,1,22.0,0.0,TAUPO,TAUPO
4,ARA2201,ARA0,2014-01-01,2,4,4.0,94.0,TAUPO,TAUPO


In [6]:
offers = offers[offers.DollarsPerMegawattHour<=300]
offers.DollarsPerMegawattHour.describe()

count    1.697680e+07
mean     5.821249e+01
std      7.539990e+01
min      0.000000e+00
25%      1.000000e-02
50%      2.162000e+01
75%      9.401000e+01
max      3.000000e+02
Name: DollarsPerMegawattHour, dtype: float64

In [7]:
offers['MWtimesdollars'] = offers['Megawatt']*offers['DollarsPerMegawattHour']
offers.head()

,PointOfConnection,Unit,TradingDate,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2,MWtimesdollars
0,ARA2201,ARA0,2014-01-01,1,1,22.0,0.0,TAUPO,TAUPO,0.0
1,ARA2201,ARA0,2014-01-01,1,4,4.0,94.0,TAUPO,TAUPO,376.0
2,ARA2201,ARA0,2014-01-01,1,5,52.0,165.0,TAUPO,TAUPO,8580.0
3,ARA2201,ARA0,2014-01-01,2,1,22.0,0.0,TAUPO,TAUPO,0.0
4,ARA2201,ARA0,2014-01-01,2,4,4.0,94.0,TAUPO,TAUPO,376.0


In [8]:
offers.set_index(['NewPointOfConnection','Unit','TradingDate','TradingPeriod','Tranche'],inplace=True)
offers.head()

PointOfConnection  Megawatt  DollarsPerMegawattHour NewPointOfConnection2  MWtimesdollars
NewPointOfConnection Unit TradingDate TradingPeriod Tranche                                                                                          
TAUPO                ARA0 2014-01-01  1             1                 ARA2201      22.0                     0.0                 TAUPO             0.0
                                                    4                 ARA2201       4.0                    94.0                 TAUPO           376.0
                                                    5                 ARA2201      52.0                   165.0                 TAUPO          8580.0
                                      2             1                 ARA2201      22.0                     0.0                 TAUPO             0.0
                                                    4                 ARA2201       4.0                    94.0                 TAUPO           376.0

In [9]:
temp1 = pd.DataFrame(offers.groupby(level=[0,2,3])['MWtimesdollars'].sum())
temp2 = pd.DataFrame(offers.groupby(level=[0,2,3])['Megawatt'].sum())
temp3 = temp1.join(temp2)
temp3.head()

MWtimesdollars  Megawatt
NewPointOfConnection TradingDate TradingPeriod                          
CYD                  2014-01-01  1                    12286.81     395.0
                                 2                    12286.81     395.0
                                 3                    12286.81     395.0
                                 4                    12286.81     395.0
                                 5                    12286.81     395.0

In [10]:
offers = offers.reset_index().set_index(['NewPointOfConnection2','Unit','TradingDate','TradingPeriod','Tranche'])
offers.head()

NewPointOfConnection PointOfConnection  Megawatt  DollarsPerMegawattHour  MWtimesdollars
NewPointOfConnection2 Unit TradingDate TradingPeriod Tranche                                                                                         
TAUPO                 ARA0 2014-01-01  1             1                      TAUPO           ARA2201      22.0                     0.0             0.0
                                                     4                      TAUPO           ARA2201       4.0                    94.0           376.0
                                                     5                      TAUPO           ARA2201      52.0                   165.0          8580.0
                                       2             1                      TAUPO           ARA2201      22.0                     0.0             0.0
                                                     4                      TAUPO           ARA2201       4.0                    94.0           376.0

In [11]:
temp1b = pd.DataFrame(offers.groupby(level=[0,2,3])['MWtimesdollars'].sum())
temp2b = pd.DataFrame(offers.groupby(level=[0,2,3])['Megawatt'].sum())
temp3b = temp1b.join(temp2b)
temp3b.head()

MWtimesdollars  Megawatt
NewPointOfConnection2 TradingDate TradingPeriod                          
CLUTHA                2014-01-01  1                    18371.55    670.01
                                  2                    18371.55    670.01
                                  3                    18371.55    670.01
                                  4                    15971.55    645.01
                                  5                    15971.55    645.01

In [12]:
temp3['WeightstarPrice'] = temp3['MWtimesdollars']/temp3['Megawatt']
temp3b['WeightstarPrice'] = temp3b['MWtimesdollars']/temp3b['Megawatt']
temp3.head()

MWtimesdollars  Megawatt  WeightstarPrice
NewPointOfConnection TradingDate TradingPeriod                                           
CYD                  2014-01-01  1                    12286.81     395.0        31.105848
                                 2                    12286.81     395.0        31.105848
                                 3                    12286.81     395.0        31.105848
                                 4                    12286.81     395.0        31.105848
                                 5                    12286.81     395.0        31.105848

In [13]:
temp3 = temp3.reset_index().set_index(['TradingDate','TradingPeriod'])
temp3 = temp3[['NewPointOfConnection','WeightstarPrice']]
temp3.head()

NewPointOfConnection  WeightstarPrice
TradingDate TradingPeriod                                      
2014-01-01  1                              CYD        31.105848
            2                              CYD        31.105848
            3                              CYD        31.105848
            4                              CYD        31.105848
            5                              CYD        31.105848

In [14]:
temp3b = temp3b.reset_index().set_index(['TradingDate','TradingPeriod'])
temp3b = temp3b[['NewPointOfConnection2','WeightstarPrice']]
temp3b.head()

NewPointOfConnection2  WeightstarPrice
TradingDate TradingPeriod                                       
2014-01-01  1                            CLUTHA        27.419815
            2                            CLUTHA        27.419815
            3                            CLUTHA        27.419815
            4                            CLUTHA        24.761709
            5                            CLUTHA        24.761709

In [15]:
allthermal = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['HUNTLY','STRATFORD','MKE','WHI','JRD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
#allgasthermal = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['HUNTLY','MKE','STRATFORD','JRD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
#do gas excluding huntly
allgasthermal = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['MKE','STRATFORD','JRD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
#do gas excluding just rankines at huntly?
#is this ok? (cos think should be grouping by whole node for computing weightstarprice)
allgasthermal_v2 = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['E3P','HLY6','TCC','PEAKERS','MKE','JRD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
huntly = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['HUNTLY'])].groupby(level=[0,1])['WeightstarPrice'].mean())
stratford = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['STRATFORD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
mckee = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['MKE'])].groupby(level=[0,1])['WeightstarPrice'].mean())
whirinaki = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['WHI'])].groupby(level=[0,1])['WeightstarPrice'].mean())
junctionrd = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['JRD'])].groupby(level=[0,1])['WeightstarPrice'].mean())
rankines = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['RANKINES'])].groupby(level=[0,1])['WeightstarPrice'].mean())
e3p = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['E3P'])].groupby(level=[0,1])['WeightstarPrice'].mean())
huntly_unit6 = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['HLY6'])].groupby(level=[0,1])['WeightstarPrice'].mean())
stratford_peakers = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['PEAKERS'])].groupby(level=[0,1])['WeightstarPrice'].mean())
TCC = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['TCC'])].groupby(level=[0,1])['WeightstarPrice'].mean())
allhydro = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['MAN','WAITAKI','CLUTHA','TKA','TAUPO','WAIKAREMOANA'])].groupby(level=[0,1])['WeightstarPrice'].mean())
allSIhydro = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['MAN','WAITAKI','CLUTHA','TKA'])].groupby(level=[0,1])['WeightstarPrice'].mean())
allNIhydro = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['TAUPO','WAIKAREMOANA','TONGARIRO'])].groupby(level=[0,1])['WeightstarPrice'].mean())
waitaki = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['WAITAKI'])].groupby(level=[0,1])['WeightstarPrice'].mean())
waikato = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['TAUPO'])].groupby(level=[0,1])['WeightstarPrice'].mean())
clutha = pd.DataFrame(temp3b[temp3b.NewPointOfConnection2.isin(['CLUTHA'])].groupby(level=[0,1])['WeightstarPrice'].mean())
tekapo = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['TKA'])].groupby(level=[0,1])['WeightstarPrice'].mean())
manapouri = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['MAN'])].groupby(level=[0,1])['WeightstarPrice'].mean())
waikaremoana = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['WAIKAREMOANA'])].groupby(level=[0,1])['WeightstarPrice'].mean())
tongariro = pd.DataFrame(temp3[temp3.NewPointOfConnection.isin(['TONGARIRO'])].groupby(level=[0,1])['WeightstarPrice'].mean())

In [16]:
#sometimes there are zero megawatts for any tranche.
allthermal = allthermal.fillna(0)
allgasthermal = allgasthermal.fillna(0)
allgasthermal_v2 = allgasthermal_v2.fillna(0)
huntly = huntly.fillna(0)
stratford = stratford.fillna(0)
mckee = mckee.fillna(0)
whirinaki = whirinaki.fillna(0)
junctionrd = junctionrd.fillna(0)
rankines = rankines.fillna(0)
e3p = e3p.fillna(0)
huntly_unit6 = huntly_unit6.fillna(0)
stratford_peakers = stratford_peakers.fillna(0)
TCC = TCC.fillna(0)
allhydro = allhydro.fillna(0)
allSIhydro = allSIhydro.fillna(0)
allNIhydro = allNIhydro.fillna(0)
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
clutha = clutha.fillna(0)
tekapo = tekapo.fillna(0)
manapouri = manapouri.fillna(0)
waikaremoana = waikaremoana.fillna(0)
tongariro = tongariro.fillna(0)

In [17]:
#save parquet files
allthermal_table = pa.Table.from_pandas(allthermal)
allgasthermal_table = pa.Table.from_pandas(allgasthermal)
allgasthermal_v2_table = pa.Table.from_pandas(allgasthermal_v2)
huntly_table = pa.Table.from_pandas(huntly)
stratford_table = pa.Table.from_pandas(stratford)
mckee_table = pa.Table.from_pandas(mckee)
whirinaki_table = pa.Table.from_pandas(whirinaki)
junctionrd_table = pa.Table.from_pandas(junctionrd)
rankines_table = pa.Table.from_pandas(rankines)
e3p_table = pa.Table.from_pandas(e3p)
huntly_unit6_table = pa.Table.from_pandas(huntly_unit6)
stratford_peakers_table = pa.Table.from_pandas(stratford_peakers)
TCC_table = pa.Table.from_pandas(TCC)
allhydro_table = pa.Table.from_pandas(allhydro)
allSIhydro_table = pa.Table.from_pandas(allSIhydro)
allNIhydro_table = pa.Table.from_pandas(allNIhydro)
waitaki_table = pa.Table.from_pandas(waitaki)
waikato_table = pa.Table.from_pandas(waikato)
clutha_table = pa.Table.from_pandas(clutha)
tekapo_table = pa.Table.from_pandas(tekapo)
manapouri_table = pa.Table.from_pandas(manapouri)
waikaremoana_table = pa.Table.from_pandas(waikaremoana)
tongariro_table = pa.Table.from_pandas(tongariro)

In [18]:
pq.write_table(allthermal_table, path + "allthermal_QWOPexc_v2.parquet")
pq.write_table(allgasthermal_table, path + "allgasthermal_QWOPexc_v2.parquet")
pq.write_table(allgasthermal_v2_table, path + "allgasthermal_QWOPexc_v2_v2.parquet")
pq.write_table(huntly_table, path + "huntly_QWOPexc_v2.parquet")
pq.write_table(stratford_table, path + "stratford_QWOPexc_v2.parquet")
pq.write_table(mckee_table, path + "mckee_QWOPexc_v2.parquet")
pq.write_table(whirinaki_table, path + "whirinaki_QWOPexc_v2.parquet")
pq.write_table(junctionrd_table, path + "junctionrd_QWOPexc_v2.parquet")
pq.write_table(rankines_table, path + "rankines_QWOPexc_v2.parquet")
pq.write_table(e3p_table, path + "e3p_QWOPexc_v2.parquet")
pq.write_table(huntly_unit6_table, path + "huntly_unit6_QWOPexc_v2.parquet")
pq.write_table(stratford_peakers_table, path + "stratford_peakers_QWOPexc_v2.parquet")
pq.write_table(TCC_table, path + "TCC_QWOPexc_v2.parquet")
pq.write_table(allhydro_table, path + "allhydro_QWOPexc_v2.parquet")
pq.write_table(allSIhydro_table, path + "allSIhydro_QWOPexc_v2.parquet")
pq.write_table(allNIhydro_table, path + "allNIhydro_QWOPexc_v2.parquet")
pq.write_table(waitaki_table, path + "waitaki_QWOPexc_v2.parquet")
pq.write_table(waikato_table, path + "waikato_QWOPexc_v2.parquet")
pq.write_table(clutha_table, path + "clutha_QWOPexc_v2.parquet")
pq.write_table(tekapo_table, path + "tekapo_QWOPexc_v2.parquet")
pq.write_table(manapouri_table, path + "manapouri_QWOPexc_v2.parquet")
pq.write_table(waikaremoana_table, path + "waikaremoana_QWOPexc_v2.parquet")
pq.write_table(tongariro_table, path + "tongariro_QWOPexc_v2.parquet")

# create parquet files with percents (over final price, SRMC, water value, fwd price)

In [ ]:
# percent greater than final price

In [5]:
offers = pd.read_parquet(path+'/rogersdata.parquet')
offers.head()

,PointOfConnection,Unit,TradingDate,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2
0,ARA2201,ARA0,2014-01-01,1,1,22.0,0.0,TAUPO,TAUPO
1,ARA2201,ARA0,2014-01-01,1,4,4.0,94.0,TAUPO,TAUPO
2,ARA2201,ARA0,2014-01-01,1,5,52.0,165.0,TAUPO,TAUPO
3,ARA2201,ARA0,2014-01-01,2,1,22.0,0.0,TAUPO,TAUPO
4,ARA2201,ARA0,2014-01-01,2,4,4.0,94.0,TAUPO,TAUPO


In [3]:
#match in final price
#Q=r"""
#SELECT [TradingDate] 
#      ,[TradingPeriod] 
#      ,[PointOfConnection] 
#      ,[DollarsPerMegawattHour] as Price
#FROM [ElectricityAuthorityData].[wholesale].[JustFinalEnergyPrices]
#WHERE TradingDate>='2014-01-01' and TradingDate<='2021-06-30'
#order by TradingDate, TradingPeriod, PointOfConnection
#"""
#df2= db.query(Q)
#df2['TradingDate'] = pd.to_datetime(df2.TradingDate)
#df2.head()

,TradingDate,TradingPeriod,PointOfConnection,Price
0,2014-01-01,1,ABY0111,33.49
1,2014-01-01,1,ADD0111,36.29
2,2014-01-01,1,ADD0661,36.23
3,2014-01-01,1,ALB0331,51.14
4,2014-01-01,1,ALB1101,51.17


In [4]:
#df2_table = pa.Table.from_pandas(df2)
#pq.write_table(df2_table, path + "df2.parquet")

In [6]:
df2 = pd.read_parquet(path + "df2.parquet")
df2.set_index(['TradingDate','TradingPeriod','PointOfConnection'],inplace=True)
df2.head()

Price
TradingDate TradingPeriod PointOfConnection       
2014-01-01  1             ABY0111            33.49
                          ADD0111            36.29
                          ADD0661            36.23
                          ALB0331            51.14
                          ALB1101            51.17

In [7]:
offers = offers.reset_index().set_index(['TradingDate','TradingPeriod','PointOfConnection'])
offers.head()

index  Unit  Tranche  Megawatt  DollarsPerMegawattHour NewPointOfConnection NewPointOfConnection2
TradingDate TradingPeriod PointOfConnection                                                                                                   
2014-01-01  1             ARA2201                0  ARA0        1      22.0                     0.0                TAUPO                 TAUPO
                          ARA2201                1  ARA0        4       4.0                    94.0                TAUPO                 TAUPO
                          ARA2201                2  ARA0        5      52.0                   165.0                TAUPO                 TAUPO
            2             ARA2201                3  ARA0        1      22.0                     0.0                TAUPO                 TAUPO
                          ARA2201                4  ARA0        4       4.0                    94.0                TAUPO                 TAUPO

In [8]:
offers = offers.join(df2)
offers.head()

index  Unit  Tranche  Megawatt  DollarsPerMegawattHour NewPointOfConnection NewPointOfConnection2  Price
TradingDate TradingPeriod PointOfConnection                                                                                                           
2014-01-01  1             ARA2201                 0  ARA0        1      22.0                    0.00                TAUPO                 TAUPO  48.09
                          ARA2201                 1  ARA0        4       4.0                   94.00                TAUPO                 TAUPO  48.09
                          ARA2201                 2  ARA0        5      52.0                  165.00                TAUPO                 TAUPO  48.09
                          ARG1101            283165  BRR0        1       6.0                    0.01                 None                  None  37.14
                          ARG1101            283166  BRR0        4       5.5                   85.00                 None                  None  37.14

In [ ]:
huntly = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['HUNTLY']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
stratford = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['STRATFORD']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
waitaki = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['WAITAKI']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
waikato = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TAUPO']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
clutha = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['CLUTHA']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
tekapo = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TKA']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())

mckee = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['MKE']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
whirinaki = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['WHI']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
junctionrd = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['JRD']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
rankines = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['RANKINES']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
e3p = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['E3P']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
huntly_unit6 = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['HLY6']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
stratford_peakers = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['PEAKERS']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())
TCC = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['TCC']))&(offers.DollarsPerMegawattHour>offers.Price)].groupby(level=[0,1])['Megawatt'].sum())

In [ ]:
huntly_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['HUNTLY'])].groupby(level=[0,1])['Megawatt'].sum())
huntly_tot = huntly_tot.rename(columns={'Megawatt':'TotMW'})
stratford_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['STRATFORD'])].groupby(level=[0,1])['Megawatt'].sum())
stratford_tot = stratford_tot.rename(columns={'Megawatt':'TotMW'})
waitaki_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['WAITAKI'])].groupby(level=[0,1])['Megawatt'].sum())
waitaki_tot = waitaki_tot.rename(columns={'Megawatt':'TotMW'})
waikato_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TAUPO'])].groupby(level=[0,1])['Megawatt'].sum())
waikato_tot = waikato_tot.rename(columns={'Megawatt':'TotMW'})
clutha_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['CLUTHA'])].groupby(level=[0,1])['Megawatt'].sum())
clutha_tot = clutha_tot.rename(columns={'Megawatt':'TotMW'})
tekapo_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TKA'])].groupby(level=[0,1])['Megawatt'].sum())
tekapo_tot = tekapo_tot.rename(columns={'Megawatt':'TotMW'})

mckee_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['MKE'])].groupby(level=[0,1])['Megawatt'].sum())
mckee_tot = mckee_tot.rename(columns={'Megawatt':'TotMW'})
whirinaki_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['WHI'])].groupby(level=[0,1])['Megawatt'].sum())
whirinaki_tot = whirinaki_tot.rename(columns={'Megawatt':'TotMW'})
junctionrd_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['JRD'])].groupby(level=[0,1])['Megawatt'].sum())
junctionrd_tot = junctionrd_tot.rename(columns={'Megawatt':'TotMW'})
rankines_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['RANKINES'])].groupby(level=[0,1])['Megawatt'].sum())
rankines_tot = rankines_tot.rename(columns={'Megawatt':'TotMW'})
e3p_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['E3P'])].groupby(level=[0,1])['Megawatt'].sum())
e3p_tot = e3p_tot.rename(columns={'Megawatt':'TotMW'})
huntly_unit6_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['HLY6'])].groupby(level=[0,1])['Megawatt'].sum())
huntly_unit6_tot = huntly_unit6_tot.rename(columns={'Megawatt':'TotMW'})
stratford_peakers_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['PEAKERS'])].groupby(level=[0,1])['Megawatt'].sum())
stratford_peakers_tot = stratford_peakers_tot.rename(columns={'Megawatt':'TotMW'})
TCC_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['TCC'])].groupby(level=[0,1])['Megawatt'].sum())
TCC_tot = TCC_tot.rename(columns={'Megawatt':'TotMW'})

In [ ]:
#need to add how='right' to pick up the times when there was MWs offered but zero MWs over the threshold
huntly = huntly.join(huntly_tot,how='right')
stratford = stratford.join(stratford_tot,how='right')
waitaki = waitaki.join(waitaki_tot,how='right')
waikato = waikato.join(waikato_tot,how='right')
clutha = clutha.join(clutha_tot,how='right')
tekapo = tekapo.join(tekapo_tot,how='right')

mckee = mckee.join(mckee_tot,how='right')
whirinaki = whirinaki.join(whirinaki_tot,how='right')
junctionrd = junctionrd.join(junctionrd_tot,how='right')
rankines = rankines.join(rankines_tot,how='right')
e3p = e3p.join(e3p_tot,how='right')
huntly_unit6 = huntly_unit6.join(huntly_unit6_tot,how='right')
stratford_peakers = stratford_peakers.join(stratford_peakers_tot,how='right')
TCC = TCC.join(TCC_tot,how='right')

In [ ]:
huntly = huntly.fillna(0)
stratford = stratford.fillna(0)
mckee = mckee.fillna(0)
whirinaki = whirinaki.fillna(0)
junctionrd = junctionrd.fillna(0)
rankines = rankines.fillna(0)
e3p = e3p.fillna(0)
huntly_unit6 = huntly_unit6.fillna(0)
stratford_peakers = stratford_peakers.fillna(0)
TCC = TCC.fillna(0)
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
clutha = clutha.fillna(0)
tekapo = tekapo.fillna(0)

In [ ]:
huntly['percent'] = (huntly['Megawatt']/huntly['TotMW'])*100
stratford['percent'] = (stratford['Megawatt']/stratford['TotMW'])*100
waitaki['percent'] = (waitaki['Megawatt']/waitaki['TotMW'])*100
waikato['percent'] = (waikato['Megawatt']/waikato['TotMW'])*100
clutha['percent'] = (clutha['Megawatt']/clutha['TotMW'])*100
tekapo['percent'] = (tekapo['Megawatt']/tekapo['TotMW'])*100

mckee['percent'] = (mckee['Megawatt']/mckee['TotMW'])*100
whirinaki['percent'] = (whirinaki['Megawatt']/whirinaki['TotMW'])*100
junctionrd['percent'] = (junctionrd['Megawatt']/junctionrd['TotMW'])*100
rankines['percent'] = (rankines['Megawatt']/rankines['TotMW'])*100
e3p['percent'] = (e3p['Megawatt']/e3p['TotMW'])*100
huntly_unit6['percent'] = (huntly_unit6['Megawatt']/huntly_unit6['TotMW'])*100
stratford_peakers['percent'] = (stratford_peakers['Megawatt']/stratford_peakers['TotMW'])*100
TCC['percent'] = (TCC['Megawatt']/TCC['TotMW'])*100

In [ ]:
#by doing this, this means the percent variable is zero if
#they have no offers above final price in that TP
huntly = huntly.fillna(0)
stratford = stratford.fillna(0)
mckee = mckee.fillna(0)
whirinaki = whirinaki.fillna(0)
junctionrd = junctionrd.fillna(0)
rankines = rankines.fillna(0)
e3p = e3p.fillna(0)
huntly_unit6 = huntly_unit6.fillna(0)
stratford_peakers = stratford_peakers.fillna(0)
TCC = TCC.fillna(0)
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
clutha = clutha.fillna(0)
tekapo = tekapo.fillna(0)

In [ ]:
#save parquet files
huntly_table = pa.Table.from_pandas(huntly)
stratford_table = pa.Table.from_pandas(stratford)
mckee_table = pa.Table.from_pandas(mckee)
whirinaki_table = pa.Table.from_pandas(whirinaki)
junctionrd_table = pa.Table.from_pandas(junctionrd)
rankines_table = pa.Table.from_pandas(rankines)
e3p_table = pa.Table.from_pandas(e3p)
huntly_unit6_table = pa.Table.from_pandas(huntly_unit6)
stratford_peakers_table = pa.Table.from_pandas(stratford_peakers)
TCC_table = pa.Table.from_pandas(TCC)
waitaki_table = pa.Table.from_pandas(waitaki)
waikato_table = pa.Table.from_pandas(waikato)
clutha_table = pa.Table.from_pandas(clutha)
tekapo_table = pa.Table.from_pandas(tekapo)

In [ ]:
pq.write_table(huntly_table, path + "huntly_percentprice_v2.parquet")
pq.write_table(stratford_table, path + "stratford_percentprice_v2.parquet")
pq.write_table(mckee_table, path + "mckee_percentprice_v2.parquet")
pq.write_table(whirinaki_table, path + "whirinaki_percentprice_v2.parquet")
pq.write_table(junctionrd_table, path + "junctionrd_percentprice_v2.parquet")
pq.write_table(rankines_table, path + "rankines_percentprice_v2.parquet")
pq.write_table(e3p_table, path + "e3p_percentprice_v2.parquet")
pq.write_table(huntly_unit6_table, path + "huntly_unit6_percentprice_v2.parquet")
pq.write_table(stratford_peakers_table, path + "stratford_peakers_percentprice_v2.parquet")
pq.write_table(TCC_table, path + "TCC_percentprice_v2.parquet")
pq.write_table(waitaki_table, path + "waitaki_percentprice_v2.parquet")
pq.write_table(waikato_table, path + "waikato_percentprice_v2.parquet")
pq.write_table(clutha_table, path + "clutha_percentprice_v2.parquet")
pq.write_table(tekapo_table, path + "tekapo_percentprice_v2.parquet")

In [ ]:
# percent greater than 300

In [3]:
offers = pd.read_parquet(path+'/rogersdata.parquet')
offers.head()

,PointOfConnection,Unit,TradingDate,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2
0,ARA2201,ARA0,2014-01-01,1,1,22.0,0.0,TAUPO,TAUPO
1,ARA2201,ARA0,2014-01-01,1,4,4.0,94.0,TAUPO,TAUPO
2,ARA2201,ARA0,2014-01-01,1,5,52.0,165.0,TAUPO,TAUPO
3,ARA2201,ARA0,2014-01-01,2,1,22.0,0.0,TAUPO,TAUPO
4,ARA2201,ARA0,2014-01-01,2,4,4.0,94.0,TAUPO,TAUPO


In [4]:
offers = offers.reset_index().set_index(['TradingDate','TradingPeriod'])

In [5]:
huntly = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['HUNTLY']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
stratford = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['STRATFORD']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
waitaki = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['WAITAKI']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
waikato = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TAUPO']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
clutha = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['CLUTHA']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
tekapo = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TKA']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())

mckee = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['MKE']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
whirinaki = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['WHI']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
junctionrd = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['JRD']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
rankines = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['RANKINES']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
e3p = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['E3P']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
huntly_unit6 = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['HLY6']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
stratford_peakers = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['PEAKERS']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())
TCC = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['TCC']))&(offers.DollarsPerMegawattHour>300)].groupby(level=[0,1])['Megawatt'].sum())

In [6]:
huntly_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['HUNTLY'])].groupby(level=[0,1])['Megawatt'].sum())
huntly_tot = huntly_tot.rename(columns={'Megawatt':'TotMW'})
stratford_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['STRATFORD'])].groupby(level=[0,1])['Megawatt'].sum())
stratford_tot = stratford_tot.rename(columns={'Megawatt':'TotMW'})
waitaki_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['WAITAKI'])].groupby(level=[0,1])['Megawatt'].sum())
waitaki_tot = waitaki_tot.rename(columns={'Megawatt':'TotMW'})
waikato_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TAUPO'])].groupby(level=[0,1])['Megawatt'].sum())
waikato_tot = waikato_tot.rename(columns={'Megawatt':'TotMW'})
clutha_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['CLUTHA'])].groupby(level=[0,1])['Megawatt'].sum())
clutha_tot = clutha_tot.rename(columns={'Megawatt':'TotMW'})
tekapo_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TKA'])].groupby(level=[0,1])['Megawatt'].sum())
tekapo_tot = tekapo_tot.rename(columns={'Megawatt':'TotMW'})

mckee_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['MKE'])].groupby(level=[0,1])['Megawatt'].sum())
mckee_tot = mckee_tot.rename(columns={'Megawatt':'TotMW'})
whirinaki_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['WHI'])].groupby(level=[0,1])['Megawatt'].sum())
whirinaki_tot = whirinaki_tot.rename(columns={'Megawatt':'TotMW'})
junctionrd_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['JRD'])].groupby(level=[0,1])['Megawatt'].sum())
junctionrd_tot = junctionrd_tot.rename(columns={'Megawatt':'TotMW'})
rankines_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['RANKINES'])].groupby(level=[0,1])['Megawatt'].sum())
rankines_tot = rankines_tot.rename(columns={'Megawatt':'TotMW'})
e3p_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['E3P'])].groupby(level=[0,1])['Megawatt'].sum())
e3p_tot = e3p_tot.rename(columns={'Megawatt':'TotMW'})
huntly_unit6_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['HLY6'])].groupby(level=[0,1])['Megawatt'].sum())
huntly_unit6_tot = huntly_unit6_tot.rename(columns={'Megawatt':'TotMW'})
stratford_peakers_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['PEAKERS'])].groupby(level=[0,1])['Megawatt'].sum())
stratford_peakers_tot = stratford_peakers_tot.rename(columns={'Megawatt':'TotMW'})
TCC_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['TCC'])].groupby(level=[0,1])['Megawatt'].sum())
TCC_tot = TCC_tot.rename(columns={'Megawatt':'TotMW'})

In [7]:
huntly = huntly.join(huntly_tot,how='right')
stratford = stratford.join(stratford_tot,how='right')
waitaki = waitaki.join(waitaki_tot,how='right')
waikato = waikato.join(waikato_tot,how='right')
clutha = clutha.join(clutha_tot,how='right')
tekapo = tekapo.join(tekapo_tot,how='right')

mckee = mckee.join(mckee_tot,how='right')
whirinaki = whirinaki.join(whirinaki_tot,how='right')
junctionrd = junctionrd.join(junctionrd_tot,how='right')
rankines = rankines.join(rankines_tot,how='right')
e3p = e3p.join(e3p_tot,how='right')
huntly_unit6 = huntly_unit6.join(huntly_unit6_tot,how='right')
stratford_peakers = stratford_peakers.join(stratford_peakers_tot,how='right')
TCC = TCC.join(TCC_tot,how='right')

In [8]:
huntly = huntly.fillna(0)
stratford = stratford.fillna(0)
mckee = mckee.fillna(0)
whirinaki = whirinaki.fillna(0)
junctionrd = junctionrd.fillna(0)
rankines = rankines.fillna(0)
e3p = e3p.fillna(0)
huntly_unit6 = huntly_unit6.fillna(0)
stratford_peakers = stratford_peakers.fillna(0)
TCC = TCC.fillna(0)
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
clutha = clutha.fillna(0)
tekapo = tekapo.fillna(0)

In [9]:
huntly['percent'] = (huntly['Megawatt']/huntly['TotMW'])*100
stratford['percent'] = (stratford['Megawatt']/stratford['TotMW'])*100
waitaki['percent'] = (waitaki['Megawatt']/waitaki['TotMW'])*100
waikato['percent'] = (waikato['Megawatt']/waikato['TotMW'])*100
clutha['percent'] = (clutha['Megawatt']/clutha['TotMW'])*100
tekapo['percent'] = (tekapo['Megawatt']/tekapo['TotMW'])*100

mckee['percent'] = (mckee['Megawatt']/mckee['TotMW'])*100
whirinaki['percent'] = (whirinaki['Megawatt']/whirinaki['TotMW'])*100
junctionrd['percent'] = (junctionrd['Megawatt']/junctionrd['TotMW'])*100
rankines['percent'] = (rankines['Megawatt']/rankines['TotMW'])*100
e3p['percent'] = (e3p['Megawatt']/e3p['TotMW'])*100
huntly_unit6['percent'] = (huntly_unit6['Megawatt']/huntly_unit6['TotMW'])*100
stratford_peakers['percent'] = (stratford_peakers['Megawatt']/stratford_peakers['TotMW'])*100
TCC['percent'] = (TCC['Megawatt']/TCC['TotMW'])*100

In [10]:
huntly = huntly.fillna(0)
stratford = stratford.fillna(0)
mckee = mckee.fillna(0)
whirinaki = whirinaki.fillna(0)
junctionrd = junctionrd.fillna(0)
rankines = rankines.fillna(0)
e3p = e3p.fillna(0)
huntly_unit6 = huntly_unit6.fillna(0)
stratford_peakers = stratford_peakers.fillna(0)
TCC = TCC.fillna(0)
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
clutha = clutha.fillna(0)
tekapo = tekapo.fillna(0)

In [11]:
#save parquet files
huntly_table = pa.Table.from_pandas(huntly)
stratford_table = pa.Table.from_pandas(stratford)
mckee_table = pa.Table.from_pandas(mckee)
whirinaki_table = pa.Table.from_pandas(whirinaki)
junctionrd_table = pa.Table.from_pandas(junctionrd)
rankines_table = pa.Table.from_pandas(rankines)
e3p_table = pa.Table.from_pandas(e3p)
huntly_unit6_table = pa.Table.from_pandas(huntly_unit6)
stratford_peakers_table = pa.Table.from_pandas(stratford_peakers)
TCC_table = pa.Table.from_pandas(TCC)
waitaki_table = pa.Table.from_pandas(waitaki)
waikato_table = pa.Table.from_pandas(waikato)
clutha_table = pa.Table.from_pandas(clutha)
tekapo_table = pa.Table.from_pandas(tekapo)

In [12]:
pq.write_table(huntly_table, path + "huntly_percent300_v2.parquet")
pq.write_table(stratford_table, path + "stratford_percent300_v2.parquet")
pq.write_table(mckee_table, path + "mckee_percent300_v2.parquet")
pq.write_table(whirinaki_table, path + "whirinaki_percent300_v2.parquet")
pq.write_table(junctionrd_table, path + "junctionrd_percent300_v2.parquet")
pq.write_table(rankines_table, path + "rankines_percent300_v2.parquet")
pq.write_table(e3p_table, path + "e3p_percent300_v2.parquet")
pq.write_table(huntly_unit6_table, path + "huntly_unit6_percent300_v2.parquet")
pq.write_table(stratford_peakers_table, path + "stratford_peakers_percent300_v2.parquet")
pq.write_table(TCC_table, path + "TCC_percent300_v2.parquet")
pq.write_table(waitaki_table, path + "waitaki_percent300_v2.parquet")
pq.write_table(waikato_table, path + "waikato_percent300_v2.parquet")
pq.write_table(clutha_table, path + "clutha_percent300_v2.parquet")
pq.write_table(tekapo_table, path + "tekapo_percent300_v2.parquet")

In [ ]:
# percent greater than FWD price

In [3]:
offers = pd.read_parquet(path+'/rogersdata.parquet')
offers.head()

,PointOfConnection,Unit,TradingDate,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2
0,ARA2201,ARA0,2014-01-01,1,1,22.0,0.0,TAUPO,TAUPO
1,ARA2201,ARA0,2014-01-01,1,4,4.0,94.0,TAUPO,TAUPO
2,ARA2201,ARA0,2014-01-01,1,5,52.0,165.0,TAUPO,TAUPO
3,ARA2201,ARA0,2014-01-01,2,1,22.0,0.0,TAUPO,TAUPO
4,ARA2201,ARA0,2014-01-01,2,4,4.0,94.0,TAUPO,TAUPO


In [4]:
#match in forward price (as measure of opp cost)
now = pd.datetime.now()

In [5]:
Q=r"""
SELECT [Settlement_Date],
	[Settlement_Price],
      [Expiration]
  
      
  FROM [ForwardMarkets].[Conf].[NZ_CLOS_SNAPSHOT_Electricity]
  
  --where Condition='BEN_QTR_BASE_FUT' and Settlement_Date is not null
  where Condition='BEN_QTR_BASE_FUT' and Settlement_Date is not null
  and [Settlement_Date]< "%s" 
  
  order by Settlement_Date , Expiration
    """ %(now)

warehouse= db.query(Q)
warehouse['TradingDate']=pandas.to_datetime(warehouse.Settlement_Date, dayfirst=True)
del warehouse['Settlement_Date']
warehouse = warehouse.sort_values(['TradingDate','Expiration'])
warehouse.set_index([ 'TradingDate', 'Expiration'], inplace=True)
warehouse.head()

Settlement_Price
TradingDate Expiration                  
2009-07-14  2009-12-31             26.40
            2010-03-31             45.95
            2010-06-30             80.50
            2010-09-30             96.45
            2010-12-31             44.00

In [6]:
warehouse2 = warehouse.groupby(level=0).mean()
warehouse2.head()

,Settlement_Price
TradingDate,
2009-07-14,72.153846
2009-07-15,72.130769
2009-07-16,72.130769
2009-07-17,71.703846
2009-07-20,71.703846


In [7]:
warehouse2.head(3000)

,Settlement_Price
TradingDate,
2009-07-14,72.153846
2009-07-15,72.130769
2009-07-16,72.130769
2009-07-17,71.703846
2009-07-20,71.703846
2009-07-21,71.703846
2009-07-22,71.657692
2009-07-23,71.657692
2009-07-24,71.657692


In [8]:
offers = offers.reset_index().set_index(['TradingDate'])
offers = offers.join(warehouse2,how='left')
offers.head()

,index,PointOfConnection,Unit,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2,Settlement_Price
TradingDate,,,,,,,,,,
2014-01-01,0,ARA2201,ARA0,1,1,22.0,0.0,TAUPO,TAUPO,NaN
2014-01-01,1,ARA2201,ARA0,1,4,4.0,94.0,TAUPO,TAUPO,NaN
2014-01-01,2,ARA2201,ARA0,1,5,52.0,165.0,TAUPO,TAUPO,NaN
2014-01-01,3,ARA2201,ARA0,2,1,22.0,0.0,TAUPO,TAUPO,NaN
2014-01-01,4,ARA2201,ARA0,2,4,4.0,94.0,TAUPO,TAUPO,NaN


In [9]:
offers.tail()

,index,PointOfConnection,Unit,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2,Settlement_Price
TradingDate,,,,,,,,,,
2021-06-30,20782117,WWD1103,WWD0,44,1,34.415,0.01,None,None,134.676667
2021-06-30,20782118,WWD1103,WWD0,45,1,40.545,0.01,None,None,134.676667
2021-06-30,20782119,WWD1103,WWD0,46,1,40.717,0.01,None,None,134.676667
2021-06-30,20782120,WWD1103,WWD0,47,1,37.957,0.01,None,None,134.676667
2021-06-30,20782121,WWD1103,WWD0,48,1,37.944,0.01,None,None,134.676667


In [10]:
offers = offers.reset_index().set_index(['TradingDate','TradingPeriod'])

In [11]:
huntly = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['HUNTLY']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
stratford = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['STRATFORD']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
waitaki = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['WAITAKI']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
waikato = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TAUPO']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
clutha = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['CLUTHA']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
tekapo = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TKA']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())

mckee = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['MKE']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
whirinaki = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['WHI']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
junctionrd = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['JRD']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
rankines = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['RANKINES']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
e3p = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['E3P']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
huntly_unit6 = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['HLY6']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
stratford_peakers = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['PEAKERS']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())
TCC = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['TCC']))&(offers.DollarsPerMegawattHour>offers.Settlement_Price)].groupby(level=[0,1])['Megawatt'].sum())

In [12]:
huntly_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['HUNTLY'])].groupby(level=[0,1])['Megawatt'].sum())
huntly_tot = huntly_tot.rename(columns={'Megawatt':'TotMW'})
stratford_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['STRATFORD'])].groupby(level=[0,1])['Megawatt'].sum())
stratford_tot = stratford_tot.rename(columns={'Megawatt':'TotMW'})
waitaki_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['WAITAKI'])].groupby(level=[0,1])['Megawatt'].sum())
waitaki_tot = waitaki_tot.rename(columns={'Megawatt':'TotMW'})
waikato_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TAUPO'])].groupby(level=[0,1])['Megawatt'].sum())
waikato_tot = waikato_tot.rename(columns={'Megawatt':'TotMW'})
clutha_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['CLUTHA'])].groupby(level=[0,1])['Megawatt'].sum())
clutha_tot = clutha_tot.rename(columns={'Megawatt':'TotMW'})
tekapo_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TKA'])].groupby(level=[0,1])['Megawatt'].sum())
tekapo_tot = tekapo_tot.rename(columns={'Megawatt':'TotMW'})

mckee_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['MKE'])].groupby(level=[0,1])['Megawatt'].sum())
mckee_tot = mckee_tot.rename(columns={'Megawatt':'TotMW'})
whirinaki_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['WHI'])].groupby(level=[0,1])['Megawatt'].sum())
whirinaki_tot = whirinaki_tot.rename(columns={'Megawatt':'TotMW'})
junctionrd_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['JRD'])].groupby(level=[0,1])['Megawatt'].sum())
junctionrd_tot = junctionrd_tot.rename(columns={'Megawatt':'TotMW'})
rankines_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['RANKINES'])].groupby(level=[0,1])['Megawatt'].sum())
rankines_tot = rankines_tot.rename(columns={'Megawatt':'TotMW'})
e3p_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['E3P'])].groupby(level=[0,1])['Megawatt'].sum())
e3p_tot = e3p_tot.rename(columns={'Megawatt':'TotMW'})
huntly_unit6_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['HLY6'])].groupby(level=[0,1])['Megawatt'].sum())
huntly_unit6_tot = huntly_unit6_tot.rename(columns={'Megawatt':'TotMW'})
stratford_peakers_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['PEAKERS'])].groupby(level=[0,1])['Megawatt'].sum())
stratford_peakers_tot = stratford_peakers_tot.rename(columns={'Megawatt':'TotMW'})
TCC_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['TCC'])].groupby(level=[0,1])['Megawatt'].sum())
TCC_tot = TCC_tot.rename(columns={'Megawatt':'TotMW'})

In [13]:
huntly = huntly.join(huntly_tot,how='right')
stratford = stratford.join(stratford_tot,how='right')
waitaki = waitaki.join(waitaki_tot,how='right')
waikato = waikato.join(waikato_tot,how='right')
clutha = clutha.join(clutha_tot,how='right')
tekapo = tekapo.join(tekapo_tot,how='right')

mckee = mckee.join(mckee_tot,how='right')
whirinaki = whirinaki.join(whirinaki_tot,how='right')
junctionrd = junctionrd.join(junctionrd_tot,how='right')
rankines = rankines.join(rankines_tot,how='right')
e3p = e3p.join(e3p_tot,how='right')
huntly_unit6 = huntly_unit6.join(huntly_unit6_tot,how='right')
stratford_peakers = stratford_peakers.join(stratford_peakers_tot,how='right')
TCC = TCC.join(TCC_tot,how='right')

In [14]:
huntly = huntly.fillna(0)
stratford = stratford.fillna(0)
mckee = mckee.fillna(0)
whirinaki = whirinaki.fillna(0)
junctionrd = junctionrd.fillna(0)
rankines = rankines.fillna(0)
e3p = e3p.fillna(0)
huntly_unit6 = huntly_unit6.fillna(0)
stratford_peakers = stratford_peakers.fillna(0)
TCC = TCC.fillna(0)
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
clutha = clutha.fillna(0)
tekapo = tekapo.fillna(0)

In [15]:
huntly['percent'] = (huntly['Megawatt']/huntly['TotMW'])*100
stratford['percent'] = (stratford['Megawatt']/stratford['TotMW'])*100
waitaki['percent'] = (waitaki['Megawatt']/waitaki['TotMW'])*100
waikato['percent'] = (waikato['Megawatt']/waikato['TotMW'])*100
clutha['percent'] = (clutha['Megawatt']/clutha['TotMW'])*100
tekapo['percent'] = (tekapo['Megawatt']/tekapo['TotMW'])*100

mckee['percent'] = (mckee['Megawatt']/mckee['TotMW'])*100
whirinaki['percent'] = (whirinaki['Megawatt']/whirinaki['TotMW'])*100
junctionrd['percent'] = (junctionrd['Megawatt']/junctionrd['TotMW'])*100
rankines['percent'] = (rankines['Megawatt']/rankines['TotMW'])*100
e3p['percent'] = (e3p['Megawatt']/e3p['TotMW'])*100
huntly_unit6['percent'] = (huntly_unit6['Megawatt']/huntly_unit6['TotMW'])*100
stratford_peakers['percent'] = (stratford_peakers['Megawatt']/stratford_peakers['TotMW'])*100
TCC['percent'] = (TCC['Megawatt']/TCC['TotMW'])*100

In [16]:
waitaki.head()

Megawatt     TotMW  percent
TradingDate TradingPeriod                             
2014-01-01  1                   0.0  1356.313      0.0
            2                   0.0  1351.825      0.0
            3                   0.0  1347.055      0.0
            4                   0.0  1367.522      0.0
            5                   0.0  1369.509      0.0

In [17]:
huntly = huntly.fillna(0)
stratford = stratford.fillna(0)
mckee = mckee.fillna(0)
whirinaki = whirinaki.fillna(0)
junctionrd = junctionrd.fillna(0)
rankines = rankines.fillna(0)
e3p = e3p.fillna(0)
huntly_unit6 = huntly_unit6.fillna(0)
stratford_peakers = stratford_peakers.fillna(0)
TCC = TCC.fillna(0)
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
clutha = clutha.fillna(0)
tekapo = tekapo.fillna(0)

In [18]:
#save parquet files
huntly_table = pa.Table.from_pandas(huntly)
stratford_table = pa.Table.from_pandas(stratford)
mckee_table = pa.Table.from_pandas(mckee)
whirinaki_table = pa.Table.from_pandas(whirinaki)
junctionrd_table = pa.Table.from_pandas(junctionrd)
rankines_table = pa.Table.from_pandas(rankines)
e3p_table = pa.Table.from_pandas(e3p)
huntly_unit6_table = pa.Table.from_pandas(huntly_unit6)
stratford_peakers_table = pa.Table.from_pandas(stratford_peakers)
TCC_table = pa.Table.from_pandas(TCC)
waitaki_table = pa.Table.from_pandas(waitaki)
waikato_table = pa.Table.from_pandas(waikato)
clutha_table = pa.Table.from_pandas(clutha)
tekapo_table = pa.Table.from_pandas(tekapo)

In [19]:
pq.write_table(huntly_table, path + "huntly_percentFWD_v2.parquet")
pq.write_table(stratford_table, path + "stratford_percentFWD_v2.parquet")
pq.write_table(mckee_table, path + "mckee_percentFWD_v2.parquet")
pq.write_table(whirinaki_table, path + "whirinaki_percentFWD_v2.parquet")
pq.write_table(junctionrd_table, path + "junctionrd_percentFWD_v2.parquet")
pq.write_table(rankines_table, path + "rankines_percentFWD_v2.parquet")
pq.write_table(e3p_table, path + "e3p_percentFWD_v2.parquet")
pq.write_table(huntly_unit6_table, path + "huntly_unit6_percentFWD_v2.parquet")
pq.write_table(stratford_peakers_table, path + "stratford_peakers_percentFWD_v2.parquet")
pq.write_table(TCC_table, path + "TCC_percentFWD_v2.parquet")
pq.write_table(waitaki_table, path + "waitaki_percentFWD_v2.parquet")
pq.write_table(waikato_table, path + "waikato_percentFWD_v2.parquet")
pq.write_table(clutha_table, path + "clutha_percentFWD_v2.parquet")
pq.write_table(tekapo_table, path + "tekapo_percentFWD_v2.parquet")

In [ ]:
# percent greater than gas SRMC

In [3]:
offers = pd.read_parquet(path+'/rogersdata.parquet')
offers.head()

,PointOfConnection,Unit,TradingDate,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2
0,ARA2201,ARA0,2014-01-01,1,1,22.0,0.0,TAUPO,TAUPO
1,ARA2201,ARA0,2014-01-01,1,4,4.0,94.0,TAUPO,TAUPO
2,ARA2201,ARA0,2014-01-01,1,5,52.0,165.0,TAUPO,TAUPO
3,ARA2201,ARA0,2014-01-01,2,1,22.0,0.0,TAUPO,TAUPO
4,ARA2201,ARA0,2014-01-01,2,4,4.0,94.0,TAUPO,TAUPO


In [4]:
roger_srmc = pd.read_parquet("/media/usb/notebooks/Julia/20192020_review/SRMC.parquet")
roger_srmc = roger_srmc.reset_index()
roger_srmc = roger_srmc.rename(columns={'date':'TradingDate'})
roger_srmc['TradingDate'] = pd.to_datetime(roger_srmc.TradingDate)
roger_srmc.set_index(['TradingDate'],inplace=True)
roger_srmc.head()

,Huntly e3p U5,TCC CCGT,"Huntly Rankine U1,2,4 coal","Huntly Rankine U1,2,4 gas",SFD OCGT,McKee OCGT,Junction Rd OCGT,HLY U6 OCGT
TradingDate,,,,,,,,
2009-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
roger_srmc['2014/1/1':].head()

,Huntly e3p U5,TCC CCGT,"Huntly Rankine U1,2,4 coal","Huntly Rankine U1,2,4 gas",SFD OCGT,McKee OCGT,Junction Rd OCGT,HLY U6 OCGT
TradingDate,,,,,,,,
2014-01-01,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177
2014-02-01,49.874857,49.874857,70.209158,75.404857,63.172831,72.940929,72.940929,73.240929
2014-03-01,49.294452,49.294452,67.456063,74.549935,62.474227,72.115419,72.115419,72.415419
2014-04-01,49.444600,49.444600,65.976369,74.771100,62.654953,72.328975,72.328975,72.628975
2014-05-01,48.100903,48.100903,66.477281,72.791871,61.037614,70.417839,70.417839,70.717839


In [6]:
offers = offers.reset_index().set_index(['TradingDate'])
offers = offers.join(roger_srmc,how='left')
offers = offers.fillna(method='ffill')
offers.head()

,index,PointOfConnection,Unit,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2,Huntly e3p U5,TCC CCGT,"Huntly Rankine U1,2,4 coal","Huntly Rankine U1,2,4 gas",SFD OCGT,McKee OCGT,Junction Rd OCGT,HLY U6 OCGT
TradingDate,,,,,,,,,,,,,,,,,
2014-01-01,0,ARA2201,ARA0,1,1,22.0,0.0,TAUPO,TAUPO,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177
2014-01-01,1,ARA2201,ARA0,1,4,4.0,94.0,TAUPO,TAUPO,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177
2014-01-01,2,ARA2201,ARA0,1,5,52.0,165.0,TAUPO,TAUPO,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177
2014-01-01,3,ARA2201,ARA0,2,1,22.0,0.0,TAUPO,TAUPO,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177
2014-01-01,4,ARA2201,ARA0,2,4,4.0,94.0,TAUPO,TAUPO,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177


In [7]:
offers['maxSRMC'] = offers[['McKee OCGT','HLY U6 OCGT','Junction Rd OCGT','Huntly Rankine U1,2,4 gas','SFD OCGT','Huntly e3p U5','TCC CCGT']].max(axis=1)
offers.head()

,index,PointOfConnection,Unit,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2,Huntly e3p U5,TCC CCGT,"Huntly Rankine U1,2,4 coal","Huntly Rankine U1,2,4 gas",SFD OCGT,McKee OCGT,Junction Rd OCGT,HLY U6 OCGT,maxSRMC
TradingDate,,,,,,,,,,,,,,,,,,
2014-01-01,0,ARA2201,ARA0,1,1,22.0,0.0,TAUPO,TAUPO,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177,72.981742
2014-01-01,1,ARA2201,ARA0,1,4,4.0,94.0,TAUPO,TAUPO,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177,72.981742
2014-01-01,2,ARA2201,ARA0,1,5,52.0,165.0,TAUPO,TAUPO,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177,72.981742
2014-01-01,3,ARA2201,ARA0,2,1,22.0,0.0,TAUPO,TAUPO,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177,72.981742
2014-01-01,4,ARA2201,ARA0,2,4,4.0,94.0,TAUPO,TAUPO,48.229806,48.229806,70.846396,72.981742,61.192768,70.601177,70.601177,70.901177,72.981742


In [8]:
offers = offers.reset_index().set_index(['TradingDate','TradingPeriod'])

In [9]:
huntly = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['HUNTLY']))&(offers.DollarsPerMegawattHour>offers['maxSRMC'])].groupby(level=[0,1])['Megawatt'].sum())
stratford = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['STRATFORD']))&(offers.DollarsPerMegawattHour>offers['maxSRMC'])].groupby(level=[0,1])['Megawatt'].sum())
waitaki = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['WAITAKI']))&(offers.DollarsPerMegawattHour>offers['maxSRMC'])].groupby(level=[0,1])['Megawatt'].sum())
waikato = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TAUPO']))&(offers.DollarsPerMegawattHour>offers['maxSRMC'])].groupby(level=[0,1])['Megawatt'].sum())
clutha = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['CLUTHA']))&(offers.DollarsPerMegawattHour>offers['maxSRMC'])].groupby(level=[0,1])['Megawatt'].sum())
tekapo = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TKA']))&(offers.DollarsPerMegawattHour>offers['maxSRMC'])].groupby(level=[0,1])['Megawatt'].sum())

mckee = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['MKE']))&(offers.DollarsPerMegawattHour>offers['McKee OCGT'])].groupby(level=[0,1])['Megawatt'].sum())
whirinaki = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['WHI']))&(offers.DollarsPerMegawattHour>offers['maxSRMC'])].groupby(level=[0,1])['Megawatt'].sum())
junctionrd = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['JRD']))&(offers.DollarsPerMegawattHour>offers['Junction Rd OCGT'])].groupby(level=[0,1])['Megawatt'].sum())
rankines = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['RANKINES']))&(offers.DollarsPerMegawattHour>offers['Huntly Rankine U1,2,4 gas'])].groupby(level=[0,1])['Megawatt'].sum())
rankines_coal = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['RANKINES']))&(offers.DollarsPerMegawattHour>offers['Huntly Rankine U1,2,4 coal'])].groupby(level=[0,1])['Megawatt'].sum())
e3p = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['E3P']))&(offers.DollarsPerMegawattHour>offers['Huntly e3p U5'])].groupby(level=[0,1])['Megawatt'].sum())
huntly_unit6 = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['HLY6']))&(offers.DollarsPerMegawattHour>offers['HLY U6 OCGT'])].groupby(level=[0,1])['Megawatt'].sum())
stratford_peakers = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['PEAKERS']))&(offers.DollarsPerMegawattHour>offers['SFD OCGT'])].groupby(level=[0,1])['Megawatt'].sum())
TCC = pd.DataFrame(offers[(offers.NewPointOfConnection.isin(['TCC']))&(offers.DollarsPerMegawattHour>offers['TCC CCGT'])].groupby(level=[0,1])['Megawatt'].sum())

In [10]:
huntly_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['HUNTLY'])].groupby(level=[0,1])['Megawatt'].sum())
huntly_tot = huntly_tot.rename(columns={'Megawatt':'TotMW'})
stratford_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['STRATFORD'])].groupby(level=[0,1])['Megawatt'].sum())
stratford_tot = stratford_tot.rename(columns={'Megawatt':'TotMW'})
waitaki_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['WAITAKI'])].groupby(level=[0,1])['Megawatt'].sum())
waitaki_tot = waitaki_tot.rename(columns={'Megawatt':'TotMW'})
waikato_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TAUPO'])].groupby(level=[0,1])['Megawatt'].sum())
waikato_tot = waikato_tot.rename(columns={'Megawatt':'TotMW'})
clutha_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['CLUTHA'])].groupby(level=[0,1])['Megawatt'].sum())
clutha_tot = clutha_tot.rename(columns={'Megawatt':'TotMW'})
tekapo_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TKA'])].groupby(level=[0,1])['Megawatt'].sum())
tekapo_tot = tekapo_tot.rename(columns={'Megawatt':'TotMW'})

mckee_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['MKE'])].groupby(level=[0,1])['Megawatt'].sum())
mckee_tot = mckee_tot.rename(columns={'Megawatt':'TotMW'})
whirinaki_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['WHI'])].groupby(level=[0,1])['Megawatt'].sum())
whirinaki_tot = whirinaki_tot.rename(columns={'Megawatt':'TotMW'})
junctionrd_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['JRD'])].groupby(level=[0,1])['Megawatt'].sum())
junctionrd_tot = junctionrd_tot.rename(columns={'Megawatt':'TotMW'})
rankines_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['RANKINES'])].groupby(level=[0,1])['Megawatt'].sum())
rankines_tot = rankines_tot.rename(columns={'Megawatt':'TotMW'})
e3p_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['E3P'])].groupby(level=[0,1])['Megawatt'].sum())
e3p_tot = e3p_tot.rename(columns={'Megawatt':'TotMW'})
huntly_unit6_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['HLY6'])].groupby(level=[0,1])['Megawatt'].sum())
huntly_unit6_tot = huntly_unit6_tot.rename(columns={'Megawatt':'TotMW'})
stratford_peakers_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['PEAKERS'])].groupby(level=[0,1])['Megawatt'].sum())
stratford_peakers_tot = stratford_peakers_tot.rename(columns={'Megawatt':'TotMW'})
TCC_tot = pd.DataFrame(offers[offers.NewPointOfConnection.isin(['TCC'])].groupby(level=[0,1])['Megawatt'].sum())
TCC_tot = TCC_tot.rename(columns={'Megawatt':'TotMW'})

In [11]:
huntly = huntly.join(huntly_tot,how='right')
stratford = stratford.join(stratford_tot,how='right')
waitaki = waitaki.join(waitaki_tot,how='right')
waikato = waikato.join(waikato_tot,how='right')
clutha = clutha.join(clutha_tot,how='right')
tekapo = tekapo.join(tekapo_tot,how='right')

mckee = mckee.join(mckee_tot,how='right')
whirinaki = whirinaki.join(whirinaki_tot,how='right')
junctionrd = junctionrd.join(junctionrd_tot,how='right')
rankines = rankines.join(rankines_tot,how='right')
rankines_coal = rankines_coal.join(rankines_tot,how='right')
e3p = e3p.join(e3p_tot,how='right')
huntly_unit6 = huntly_unit6.join(huntly_unit6_tot,how='right')
stratford_peakers = stratford_peakers.join(stratford_peakers_tot,how='right')
TCC = TCC.join(TCC_tot,how='right')

In [12]:
huntly = huntly.fillna(0)
stratford = stratford.fillna(0)
mckee = mckee.fillna(0)
whirinaki = whirinaki.fillna(0)
junctionrd = junctionrd.fillna(0)
rankines = rankines.fillna(0)
rankines_coal = rankines_coal.fillna(0)
e3p = e3p.fillna(0)
huntly_unit6 = huntly_unit6.fillna(0)
stratford_peakers = stratford_peakers.fillna(0)
TCC = TCC.fillna(0)
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
clutha = clutha.fillna(0)
tekapo = tekapo.fillna(0)

In [13]:
huntly['percent'] = (huntly['Megawatt']/huntly['TotMW'])*100
stratford['percent'] = (stratford['Megawatt']/stratford['TotMW'])*100
waitaki['percent'] = (waitaki['Megawatt']/waitaki['TotMW'])*100
waikato['percent'] = (waikato['Megawatt']/waikato['TotMW'])*100
clutha['percent'] = (clutha['Megawatt']/clutha['TotMW'])*100
tekapo['percent'] = (tekapo['Megawatt']/tekapo['TotMW'])*100

mckee['percent'] = (mckee['Megawatt']/mckee['TotMW'])*100
whirinaki['percent'] = (whirinaki['Megawatt']/whirinaki['TotMW'])*100
junctionrd['percent'] = (junctionrd['Megawatt']/junctionrd['TotMW'])*100
rankines['percent'] = (rankines['Megawatt']/rankines['TotMW'])*100
rankines_coal['percent'] = (rankines_coal['Megawatt']/rankines_coal['TotMW'])*100
e3p['percent'] = (e3p['Megawatt']/e3p['TotMW'])*100
huntly_unit6['percent'] = (huntly_unit6['Megawatt']/huntly_unit6['TotMW'])*100
stratford_peakers['percent'] = (stratford_peakers['Megawatt']/stratford_peakers['TotMW'])*100
TCC['percent'] = (TCC['Megawatt']/TCC['TotMW'])*100

In [14]:
huntly = huntly.fillna(0)
stratford = stratford.fillna(0)
mckee = mckee.fillna(0)
whirinaki = whirinaki.fillna(0)
junctionrd = junctionrd.fillna(0)
rankines = rankines.fillna(0)
rankines_coal = rankines_coal.fillna(0)
e3p = e3p.fillna(0)
huntly_unit6 = huntly_unit6.fillna(0)
stratford_peakers = stratford_peakers.fillna(0)
TCC = TCC.fillna(0)
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
clutha = clutha.fillna(0)
tekapo = tekapo.fillna(0)

In [15]:
#save parquet files
huntly_table = pa.Table.from_pandas(huntly)
stratford_table = pa.Table.from_pandas(stratford)
mckee_table = pa.Table.from_pandas(mckee)
whirinaki_table = pa.Table.from_pandas(whirinaki)
junctionrd_table = pa.Table.from_pandas(junctionrd)
rankines_table = pa.Table.from_pandas(rankines)
rankines_coal_table = pa.Table.from_pandas(rankines_coal)
e3p_table = pa.Table.from_pandas(e3p)
huntly_unit6_table = pa.Table.from_pandas(huntly_unit6)
stratford_peakers_table = pa.Table.from_pandas(stratford_peakers)
TCC_table = pa.Table.from_pandas(TCC)
waitaki_table = pa.Table.from_pandas(waitaki)
waikato_table = pa.Table.from_pandas(waikato)
clutha_table = pa.Table.from_pandas(clutha)
tekapo_table = pa.Table.from_pandas(tekapo)

In [16]:
pq.write_table(huntly_table, path + "huntly_percentSRMC_v2.parquet")
pq.write_table(stratford_table, path + "stratford_percentSRMC_v2.parquet")
pq.write_table(mckee_table, path + "mckee_percentSRMC_v2.parquet")
pq.write_table(whirinaki_table, path + "whirinaki_percentSRMC_v2.parquet")
pq.write_table(junctionrd_table, path + "junctionrd_percentSRMC_v2.parquet")
pq.write_table(rankines_table, path + "rankines_percentSRMC_v2.parquet")
pq.write_table(rankines_coal_table, path + "rankines_coal_percentSRMC_v2.parquet")
pq.write_table(e3p_table, path + "e3p_percentSRMC_v2.parquet")
pq.write_table(huntly_unit6_table, path + "huntly_unit6_percentSRMC_v2.parquet")
pq.write_table(stratford_peakers_table, path + "stratford_peakers_percentSRMC_v2.parquet")
pq.write_table(TCC_table, path + "TCC_percentSRMC_v2.parquet")
pq.write_table(waitaki_table, path + "waitaki_percentSRMC_v2.parquet")
pq.write_table(waikato_table, path + "waikato_percentSRMC_v2.parquet")
pq.write_table(clutha_table, path + "clutha_percentSRMC_v2.parquet")
pq.write_table(tekapo_table, path + "tekapo_percentSRMC_v2.parquet")

In [ ]:
# percent greater than water values

In [7]:
offers = pd.read_parquet(path+'/rogersdata.parquet')
offers = offers.reset_index().set_index(['TradingDate'])
offers.head()

,index,PointOfConnection,Unit,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2
TradingDate,,,,,,,,,
2014-01-01,0,ARA2201,ARA0,1,1,22.0,0.0,TAUPO,TAUPO
2014-01-01,1,ARA2201,ARA0,1,4,4.0,94.0,TAUPO,TAUPO
2014-01-01,2,ARA2201,ARA0,1,5,52.0,165.0,TAUPO,TAUPO
2014-01-01,3,ARA2201,ARA0,2,1,22.0,0.0,TAUPO,TAUPO
2014-01-01,4,ARA2201,ARA0,2,4,4.0,94.0,TAUPO,TAUPO


In [9]:
#match in watervalues
meridian_watervalues = pd.read_parquet("/media/usb/notebooks/Julia/20192020_review/meridian_watervalues.parquet")
mercury_watervalues = pd.read_parquet("/media/usb/notebooks/Julia/20192020_review/mercury_watervalues.parquet")
genesis_watervalues = pd.read_parquet("/media/usb/notebooks/Julia/20192020_review/genesis_watervalues.parquet")

In [10]:
meridian_watervalues = meridian_watervalues.reset_index()
meridian_watervalues = meridian_watervalues.rename(columns={'Date':'TradingDate','WaterValue':'Meridian_watervalue'})
meridian_watervalues.set_index(['TradingDate'],inplace=True)
#meridian_watervalues.head()

In [11]:
mercury_watervalues = mercury_watervalues.reset_index()
mercury_watervalues = mercury_watervalues.rename(columns={'Date':'TradingDate','WaterValue':'Mercury_watervalue'})
mercury_watervalues.set_index(['TradingDate'],inplace=True)
#mercury_watervalues.head()

In [12]:
genesis_watervalues = genesis_watervalues.reset_index()
genesis_watervalues = genesis_watervalues[['Date','WeightstarPrice']]
genesis_watervalues = genesis_watervalues.rename(columns={'Date':'TradingDate','WeightstarPrice':'Genesis_watervalue'})
genesis_watervalues.set_index(['TradingDate'],inplace=True)
#genesis_watervalues.head()

In [13]:
offers = offers.join(meridian_watervalues).join(mercury_watervalues).join(genesis_watervalues)
#offers.head()

In [16]:
offers = offers.reset_index().set_index(['TradingDate','TradingPeriod'])

In [17]:
waitaki = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['WAITAKI']))&(offers.DollarsPerMegawattHour>offers['Meridian_watervalue'])].groupby(level=[0,1])['Megawatt'].sum())
waikato = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TAUPO']))&(offers.DollarsPerMegawattHour>offers['Mercury_watervalue'])].groupby(level=[0,1])['Megawatt'].sum())
tekapo = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TKA']))&(offers.DollarsPerMegawattHour>offers['Genesis_watervalue'])].groupby(level=[0,1])['Megawatt'].sum())

In [13]:
len(tekapo)

65758

In [14]:
waitaki2 = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['WAITAKI']))&(offers.DollarsPerMegawattHour>(offers['Meridian_watervalue'])+7.42)].groupby(level=[0,1])['Megawatt'].sum())
waitaki2 = waitaki2.rename(columns={'Megawatt':'MW_incSIMI'})
waikato2 = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TAUPO']))&(offers.DollarsPerMegawattHour>(offers['Mercury_watervalue']+1))].groupby(level=[0,1])['Megawatt'].sum())
waikato2 = waikato2.rename(columns={'Megawatt':'MW_incSIMI'})
tekapo2 = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TKA']))&(offers.DollarsPerMegawattHour>(offers['Genesis_watervalue']+7.42))].groupby(level=[0,1])['Megawatt'].sum())
tekapo2 = tekapo2.rename(columns={'Megawatt':'MW_incSIMI'})

In [15]:
waitaki = waitaki.join(waitaki2,how='left')
waikato = waikato.join(waikato2,how='left')
tekapo = tekapo.join(tekapo2,how='left')

In [16]:
len(tekapo)

65758

In [17]:
waitaki_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['WAITAKI'])].groupby(level=[0,1])['Megawatt'].sum())
waitaki_tot = waitaki_tot.rename(columns={'Megawatt':'TotMW'})
waikato_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TAUPO'])].groupby(level=[0,1])['Megawatt'].sum())
waikato_tot = waikato_tot.rename(columns={'Megawatt':'TotMW'})
tekapo_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TKA'])].groupby(level=[0,1])['Megawatt'].sum())
tekapo_tot = tekapo_tot.rename(columns={'Megawatt':'TotMW'})

In [18]:
len(tekapo_tot)

127933

In [19]:
waitaki = waitaki.join(waitaki_tot,how='right')
waikato = waikato.join(waikato_tot,how='right')
tekapo = tekapo.join(tekapo_tot,how='right')

In [20]:
len(tekapo)

127933

In [21]:
tekapo.head()

Megawatt  MW_incSIMI  TotMW
TradingDate TradingPeriod                             
2014-01-01  1                   NaN         NaN  150.0
            2                   NaN         NaN  150.0
            3                   NaN         NaN  150.0
            4                   NaN         NaN  150.0
            5                   NaN         NaN  150.0

In [22]:
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
tekapo = tekapo.fillna(0)

In [23]:
waitaki['percent'] = (waitaki['Megawatt']/waitaki['TotMW'])*100
waikato['percent'] = (waikato['Megawatt']/waikato['TotMW'])*100
tekapo['percent'] = (tekapo['Megawatt']/tekapo['TotMW'])*100

waitaki['percent2'] = (waitaki['MW_incSIMI']/waitaki['TotMW'])*100
waikato['percent2'] = (waikato['MW_incSIMI']/waikato['TotMW'])*100
tekapo['percent2'] = (tekapo['MW_incSIMI']/tekapo['TotMW'])*100

In [24]:
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
tekapo = tekapo.fillna(0)

In [25]:
waitaki_table = pa.Table.from_pandas(waitaki)
waikato_table = pa.Table.from_pandas(waikato)
tekapo_table = pa.Table.from_pandas(tekapo)

In [26]:
pq.write_table(waitaki_table, path + "waitaki_percentWV_v2.parquet")
pq.write_table(waikato_table, path + "waikato_percentWV_v2.parquet")
pq.write_table(tekapo_table, path + "tekapo_percentWV_v2.parquet")

In [20]:
# percent greater than DOASA water values

In [21]:
offers = pd.read_parquet(path+'/rogersdata.parquet')
offers = offers.reset_index().set_index(['TradingDate'])
offers.head()

,index,PointOfConnection,Unit,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2
TradingDate,,,,,,,,,
2014-01-01,0,ARA2201,ARA0,1,1,22.0,0.0,TAUPO,TAUPO
2014-01-01,1,ARA2201,ARA0,1,4,4.0,94.0,TAUPO,TAUPO
2014-01-01,2,ARA2201,ARA0,1,5,52.0,165.0,TAUPO,TAUPO
2014-01-01,3,ARA2201,ARA0,2,1,22.0,0.0,TAUPO,TAUPO
2014-01-01,4,ARA2201,ARA0,2,4,4.0,94.0,TAUPO,TAUPO


In [22]:
#match in watervalues
DOASA_watervalues = pd.read_parquet("/media/usb/notebooks/Julia/20192020_review/DOASA_avg_watervalue.parquet")
DOASA_watervalues = DOASA_watervalues.reset_index()
DOASA_watervalues = DOASA_watervalues.rename(columns={'Date':'TradingDate'})
DOASA_watervalues = DOASA_watervalues.set_index(['TradingDate'])
DOASA_watervalues.head()

,marginal_H20_value
TradingDate,
2016-01-01,56.11
2016-01-02,56.12
2016-01-03,56.12
2016-01-04,56.11
2016-01-05,56.12


In [23]:
offers = offers.join(DOASA_watervalues)
offers.tail()

,index,PointOfConnection,Unit,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2,marginal_H20_value
TradingDate,,,,,,,,,,
2021-06-30,20782117,WWD1103,WWD0,44,1,34.415,0.01,None,None,92.44
2021-06-30,20782118,WWD1103,WWD0,45,1,40.545,0.01,None,None,92.44
2021-06-30,20782119,WWD1103,WWD0,46,1,40.717,0.01,None,None,92.44
2021-06-30,20782120,WWD1103,WWD0,47,1,37.957,0.01,None,None,92.44
2021-06-30,20782121,WWD1103,WWD0,48,1,37.944,0.01,None,None,92.44


In [24]:
offers.head()

,index,PointOfConnection,Unit,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2,marginal_H20_value
TradingDate,,,,,,,,,,
2014-01-01,0,ARA2201,ARA0,1,1,22.0,0.0,TAUPO,TAUPO,NaN
2014-01-01,1,ARA2201,ARA0,1,4,4.0,94.0,TAUPO,TAUPO,NaN
2014-01-01,2,ARA2201,ARA0,1,5,52.0,165.0,TAUPO,TAUPO,NaN
2014-01-01,3,ARA2201,ARA0,2,1,22.0,0.0,TAUPO,TAUPO,NaN
2014-01-01,4,ARA2201,ARA0,2,4,4.0,94.0,TAUPO,TAUPO,NaN


In [25]:
offers = offers.reset_index().set_index(['TradingDate','TradingPeriod'])

In [26]:
waitaki = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['WAITAKI']))&(offers.DollarsPerMegawattHour>offers['marginal_H20_value'])].groupby(level=[0,1])['Megawatt'].sum())
waikato = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TAUPO']))&(offers.DollarsPerMegawattHour>offers['marginal_H20_value'])].groupby(level=[0,1])['Megawatt'].sum())
tekapo = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['TKA']))&(offers.DollarsPerMegawattHour>offers['marginal_H20_value'])].groupby(level=[0,1])['Megawatt'].sum())
clutha = pd.DataFrame(offers[(offers.NewPointOfConnection2.isin(['CLUTHA']))&(offers.DollarsPerMegawattHour>offers['marginal_H20_value'])].groupby(level=[0,1])['Megawatt'].sum())

In [27]:
waitaki_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['WAITAKI'])].groupby(level=[0,1])['Megawatt'].sum())
waitaki_tot = waitaki_tot.rename(columns={'Megawatt':'TotMW'})
waikato_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TAUPO'])].groupby(level=[0,1])['Megawatt'].sum())
waikato_tot = waikato_tot.rename(columns={'Megawatt':'TotMW'})
tekapo_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['TKA'])].groupby(level=[0,1])['Megawatt'].sum())
tekapo_tot = tekapo_tot.rename(columns={'Megawatt':'TotMW'})
clutha_tot = pd.DataFrame(offers[offers.NewPointOfConnection2.isin(['CLUTHA'])].groupby(level=[0,1])['Megawatt'].sum())
clutha_tot = clutha_tot.rename(columns={'Megawatt':'TotMW'})

In [28]:
waitaki = waitaki.join(waitaki_tot,how='right')
waikato = waikato.join(waikato_tot,how='right')
tekapo = tekapo.join(tekapo_tot,how='right')
clutha = clutha.join(clutha_tot,how='right')

In [29]:
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
tekapo = tekapo.fillna(0)
clutha = clutha.fillna(0)

In [30]:
waitaki['percent'] = (waitaki['Megawatt']/waitaki['TotMW'])*100
waikato['percent'] = (waikato['Megawatt']/waikato['TotMW'])*100
tekapo['percent'] = (tekapo['Megawatt']/tekapo['TotMW'])*100
clutha['percent'] = (clutha['Megawatt']/clutha['TotMW'])*100

In [31]:
waitaki.head()

Megawatt     TotMW  percent
TradingDate TradingPeriod                             
2014-01-01  1                   0.0  1356.313      0.0
            2                   0.0  1351.825      0.0
            3                   0.0  1347.055      0.0
            4                   0.0  1367.522      0.0
            5                   0.0  1369.509      0.0

In [32]:
waitaki = waitaki.fillna(0)
waikato = waikato.fillna(0)
tekapo = tekapo.fillna(0)
clutha = clutha.fillna(0)

In [33]:
waitaki_table = pa.Table.from_pandas(waitaki)
waikato_table = pa.Table.from_pandas(waikato)
tekapo_table = pa.Table.from_pandas(tekapo)
clutha_table = pa.Table.from_pandas(clutha)

In [34]:
pq.write_table(waitaki_table, path + "waitaki_percentDOASA_WV_v2.parquet")
pq.write_table(waikato_table, path + "waikato_percentDOASA_WV_v2.parquet")
pq.write_table(tekapo_table, path + "tekapo_percentDOASA_WV_v2.parquet")
pq.write_table(clutha_table, path + "clutha_percentDOASA_WV_v2.parquet")